# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [56]:
#Load csv
vac_weather = os.path.join("city_weather.csv")
vac_weather_df = pd.read_csv(vac_weather)
vac_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pacific grove,38.48,-90.74,52.00,86,1,5.61,US,1607659939
1,1,norwich,52.63,1.30,44.60,93,100,14.99,GB,1607659939
2,2,vaini,-21.20,-175.20,84.20,79,20,11.41,TO,1607659939
3,3,sainte-anne-des-monts,49.12,-66.49,28.40,79,54,9.17,CA,1607659939
4,4,flinders,-34.58,150.86,66.20,45,75,14.99,AU,1607659940
...,...,...,...,...,...,...,...,...,...,...
509,509,alikalia,9.17,-11.40,69.55,97,43,1.79,SL,1607660037
510,510,deputatskiy,69.30,139.90,-34.02,82,8,4.18,RU,1607660037
511,511,itoman,26.12,127.67,78.80,78,75,11.41,JP,1607660037
512,512,rosetown,51.55,-108.00,3.20,84,7,6.93,CA,1607660038


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [57]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [58]:
#Use Lat and Lng as locations and Humidity 
locations = vac_weather_df[["Lat","Lng"]]
humidity = vac_weather_df["Humidity"]

#heatmap map
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
#heatmap map
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:
weather_file = vac_weather_df.loc[(vac_weather_df['Max Temp'] >= 70) & (vac_weather_df['Max Temp'] >= 80)]
wind_file = weather_file.loc[weather_file['Wind Speed']<=10]
perfect_weather = wind_file.loc[wind_file['Cloudiness']==0]
ideal_weather = perfect_weather.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
#Store into variable named hotel_df.
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,176,bubaque,11.28,-15.83,80.49,65,0,6.82,GW,1607659971
1,254,santa maria,17.49,120.43,85.98,65,0,5.10,PH,1607659799
2,368,santa cruz de la palma,17.49,120.43,85.98,65,0,5.10,PH,1607660009


In [62]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,176,bubaque,11.28,-15.83,80.49,65,0,6.82,GW,1607659971,
1,254,santa maria,17.49,120.43,85.98,65,0,5.10,PH,1607659799,
2,368,santa cruz de la palma,17.49,120.43,85.98,65,0,5.10,PH,1607660009,


In [63]:
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,176,bubaque,11.28,-15.83,80.49,65,0,6.82,GW,1607659971,NaN
1,254,santa maria,17.49,120.43,85.98,65,0,5.10,PH,1607659799,NaN
2,368,santa cruz de la palma,17.49,120.43,85.98,65,0,5.10,PH,1607660009,NaN


In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [65]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))